In [79]:
# coding=utf-8
import math
from scapy.all import *
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd

#hosts = dict([('www.easternct.edu','149.152.33.32'), ('web.mit.edu','23.197.229.238'), ('http://www.ulbsibiu.ro/ro/', 
#'194.169.191.4')])
hosts = dict([('marruecos', '159.253.148.195')])

DEFAULT_RTT_VALUE = 0

ATTEMPS = 30

def enviar_paquete(host, time_to_live):
    packet = IP(dst=host, ttl=time_to_live) / ICMP()
    res = sr(packet, timeout=1, verbose=0)
    return res[0][ICMP]

In [80]:
def trace_route(hosts):
    
    hops = []
    
    for host in hosts:
        ttl_without_response = []
        ip = hosts[host]
        tipo = 11
        ttl = 1
        while (tipo!=0 and ttl < 30):
            for i in range(0,ATTEMPS):
                if (not ttl in ttl_without_response):
                    ti = time.time()
                    res1 = enviar_paquete(ip, ttl)
                    tf = time.time()
                    rtt = tf-ti
                    if (len(res1) > 0):
                        tipo = res1[0][1].type
                        src = res1[0][1].src
                        if (tipo > 0):
                            hops.append([host,ttl,i,src,rtt])
                        else:
                            hops.append([host,ttl,i,src,rtt])
                    else:
                        hops.append([host,ttl,i,None,DEFAULT_RTT_VALUE])
                        ttl_without_response.append(ttl)
            ttl = ttl + 1

    hops_info = pd.DataFrame(hops,columns=['host','ttl','i','ip','rtt'])
    return hops_info

In [81]:
df = trace_route(hosts)
df_indexed = df.set_index(['host','ttl','i'])
df_indexed

ip       rtt
host      ttl i                        
marruecos 1   0   192.168.0.1  0.091119
              1   192.168.0.1  0.045582
              2   192.168.0.1  0.035760
              3   192.168.0.1  0.038098
              4   192.168.0.1  0.038499
              5   192.168.0.1  0.056220
              6   192.168.0.1  0.046841
              7   192.168.0.1  0.039050
              8   192.168.0.1  0.038600
              9   192.168.0.1  0.039328
              10  192.168.0.1  0.042522
              11  192.168.0.1  0.052329
              12  192.168.0.1  0.055329
              13  192.168.0.1  0.038354
              14  192.168.0.1  0.043677
              15  192.168.0.1  0.042034
              16  192.168.0.1  0.037467
              17  192.168.0.1  0.044684
              18  192.168.0.1  0.038779
              19  192.168.0.1  0.039531
              20  192.168.0.1  0.039324
              21         None  0.000000
          2   0   10.21.128.1  0.072634
              1   10.21.128.1  0.050508
              2          None  0.000000
          3   0          None  0.000000
          4   0          None  0.000000
          5   0          None  0.000000
          6   0          None  0.000000
          7   0          None  0.000000
          8   0          None  0.000000
          9   0          None  0.000000
          10  0          None  0.000000
          11  0          None  0.000000
          12  0          None  0.000000
          13  0          None  0.000000
          14  0          None  0.000000
          15  0          None  0.000000
          16  0          None  0.000000
          17  0          None  0.000000
          18  0          None  0.000000
          19  0          None  0.000000
          20  0          None  0.000000
          21  0          None  0.000000
          22  0          None  0.000000
          23  0          None  0.000000
          24  0          None  0.000000
          25  0          None  0.000000
          26  0          None  0.000000
          27  0          None  0.000000
          28  0          None  0.000000
          29  0          None  0.000000

In [82]:
agrupados = df.groupby('ttl')
ips = df['ip']

for row in ips:
    print(row)


192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
192.168.0.1
None
10.21.128.1
10.21.128.1
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [87]:
ips = set(df['ip'].astype(unicode).tolist())

#type(ips[0])
", ".join(str(e) for e in ips)

u'10.21.128.1, None, 192.168.0.1'

In [20]:
import logging
import sys
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import *
dst_ip = "web.mit.edu"
src_port = 400
dst_port=80
 
tcp_connect_scan_resp = sr1(IP(dst=dst_ip)/TCP(sport=src_port,dport=dst_port,flags="S"),timeout=10)
if(tcp_connect_scan_resp is None):
    print ("The port is Closed")
    sys.exit()
     
print ("The flags:" + str (tcp_connect_scan_resp.getlayer(TCP).flags))    
    
if(tcp_connect_scan_resp.haslayer(TCP)):
    if(tcp_connect_scan_resp.getlayer(TCP).flags == 0x12):
        #send_rst = sr(IP(dst=dst_ip)/TCP(sport=src_port,dport=dst_port,flags="AR"),timeout=3)
        print ("The port is Open")
        print tcp_connect_scan_resp.getlayer(IP).src
        sys.exit ();
            
#(tcp_connect_scan_resp.getlayer(TCP).flags == 0x14):
print ("The port is Closed ")

SyntaxError: invalid syntax (<ipython-input-20-3c475ae5386b>, line 20)

In [ ]:
df.set_index?


In [4]:
sr1(IP(dst="8.8.8.8")/UDP(dport=53)/DNS(rd=1, qd=DNSQR(qname="www.google.com", qclass="IN", qtype="A")),verbose=0)

<IP  version=4L ihl=5L tos=0x0 len=316 id=45698 flags= frag=0L ttl=55 proto=udp chksum=0xff17 src=8.8.8.8 dst=192.168.0.95 options=[] |<UDP  sport=domain dport=domain len=296 chksum=0x34a6 |<DNS  id=0 qr=1L opcode=QUERY aa=0L tc=0L rd=1L ra=1L z=0L ad=0L cd=0L rcode=ok qdcount=1 ancount=16 nscount=0 arcount=0 qd=<DNSQR  qname='www.google.com.' qtype=A qclass=IN |> an=<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.246' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.217' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.216' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.236' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.212' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.242' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.226' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata=

In [5]:
import dns.resolver
my_resolver = dns.resolver.Resolver()
my_resolver.nameservers = ['8.8.8.8']

answer=my_resolver.query("google.com", "A")
for data in answer:
    print data.address

ImportError: No module named dns.resolver

In [6]:
dns.rdtypes.IN.A.A.address

NameError: name 'dns' is not defined

In [5]:
#!/usr/bin/env python
# coding: utf-8

from __future__ import print_function, unicode_literals, with_statement
import argparse
import contextlib
import requests
import sys
import csv
import matplotlib
# Anti-Grain Geometry (AGG) backend so PyGeoIpMap can be used 'headless'
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
#import pygeoip
import geoip2.database

def get_ip(ip_file):
    """
    Returns a list of IP addresses from a file containing one IP per line.
    """
    with contextlib.closing(ip_file):
        return [line.strip() for line in ip_file]


def get_lat_lon(ip_list=[], lats=[], lons=[]):
    """
    This function connects to the FreeGeoIP web service to get info from
    a list of IP addresses.
    Returns two lists (latitude and longitude).
    """
    print("Processing {} IPs...".format(len(ip_list)))
    for ip in ip_list:
        r = requests.get("https://freegeoip.net/json/" + ip)
        json_response = r.json()
        print("{ip}, {region_name}, {country_name}, {latitude}, {longitude}".format(**json_response))
        if json_response['latitude'] and json_response['longitude']:
            lats.append(json_response['latitude'])
            lons.append(json_response['longitude'])
    return lats, lons


def geoip_lat_lon(gi, ip_list=[], lats=[], lons=[]):
    """
    This function uses the MaxMind library and databases to geolocate IP addresses
    Returns two lists (latitude and longitude).
    """
    print("Processing {} IPs...".format(len(ip_list)))
    for ip in ip_list:
        try:
            r = gi.record_by_addr(ip)
        except Exception:
            print("Unable to locate IP: %s" % ip)
            continue
        if r is not None:
            print("%s {country_code} {latitude}, {longitude}".format(**r) % ip)
            lats.append(r['latitude'])
            lons.append(r['longitude'])
    return lats, lons


def get_lat_lon_from_csv(csv_file, lats=[], lons=[]):
    """
    Retrieves the last two rows of a CSV formatted file to use as latitude
    and longitude.
    Returns two lists (latitudes and longitudes).
    Example CSV file:
    119.80.39.54, Beijing, China, 39.9289, 116.3883
    101.44.1.135, Shanghai, China, 31.0456, 121.3997
    219.144.17.74, Xian, China, 34.2583, 108.9286
    64.27.26.7, Los Angeles, United States, 34.053, -118.2642
    """
    with contextlib.closing(csv_file):
        reader = csv.reader(csv_file)
        for row in reader:
            lats.append(row[-2])
            lons.append(row[-1])

    return lats, lons


def generate_map(output, lats=[], lons=[], wesn=None):
    """
    Using Basemap and the matplotlib toolkit, this function generates a map and
    puts a red dot at the location of every IP addresses found in the list.
    The map is then saved in the file specified in `output`.
    """
    
    plt.clf()
    print("Generating map and saving it to {}".format(output))
    if wesn:
        wesn = [float(i) for i in wesn.split('/')]
        m = Basemap(projection='cyl', resolution='l',
                llcrnrlon=wesn[0], llcrnrlat=wesn[2],
                urcrnrlon=wesn[1], urcrnrlat=wesn[3])
    else:
        m = Basemap(projection='cyl', resolution='l')
    m.bluemarble()
    #m.shadedrelief()
    #m.etopo()
    x, y = m(lons, lats)
    #m.pyplot(x, 30, 40, 40, color='w')
    #x1, y1 = m(lons+30, lats+30)
    #m.quiver(x, y, 30, 30, color='w')
    
    m.plot(x, y, 40, 40, color='y')
    m.scatter(x, y, s=3, color='#ff0000', marker='o', alpha=0.3) #, alpha=0.3
    #m.plot(-34.6033, -58.3816, 52.35, 4.9167, color='y', latlon=False)
    #m.plot(x, y, '-o')
    
    plt.savefig(output, dpi=300, bbox_inches='tight')







ImportError: No module named geoip2.database

In [24]:
ip_list = ['218.60.148.32', '59.63.175.24', '109.207.56.22', '59.63.175.25', '59.39.71.222', '222.186.62.17', '72.80.16.100', '60.199.196.144']
lista_rusia = ['192.168.0.1', '10.21.128.1', '100.72.1.77', '100.72.1.13', '10.242.1.194', '208.178.195.209', '212.162.45.194', '37.128.239.141', '37.128.239.110', '37.128.233.154', '217.73.167.147']
lista_marruecos = ['192.168.0.1', '10.21.128.1', '100.72.1.77', '100.72.1.14', '100.72.1.13', '10.242.1.194', '190.94.176.106', '177.84.161.238', '187.16.220.213', '50.97.19.215', '50.97.19.221', '173.192.18.176', '50.97.19.205', '50.97.18.211', '50.97.18.249', '159.253.158.131', '159.253.148.195'] 

lista_trace_nuestro = ['200.89.161.129', '200.89.165.250', '195.22.219.3', '149.3.181.65',
       '129.250.2.227', '129.250.2.19', '129.250.4.141', '129.250.6.8',
       '81.25.207.146', '50.97.18.211', '50.97.18.249', '159.253.158.131',
       '159.253.148.195']

enlace_intercontinental = ['129.250.2.227', '129.250.2.19']

#gi = pygeoip.GeoIP('GeoLiteCity.dat')


reader = geoip2.database.Reader('GeoLite2-City.mmdb')
response2 = reader.city('128.101.101.101')
print response2.country.iso_code
response2.country.name
response2.country.names['zh-CN']
response2.subdivisions.most_specific.name
response2.subdivisions.most_specific.iso_code
response2.city.name
response2.postal.code
response2.location.latitude
response2.location.longitude
reader.close()

#lats, lons = geoip_lat_lon(gi, lista_trace_nuestro)
#generate_map('salida.png', lats, lons) #, wesn='-12/45/30/65'

SyntaxError: invalid syntax (<ipython-input-24-c9af9786e24e>, line 17)

In [40]:
m = Basemap(projection='cyl', resolution='l')
m.bluemarble()
x, y = m(lons, lats)
type(x)


list

In [63]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0, 5, 0.1);
y = np.sin(x)
plt.plot(x, y)

In [25]:
#from __future__ import print_function, unicode_literals, with_statement
import argparse
import contextlib
import requests
import sys
import csv
import matplotlib
# Anti-Grain Geometry (AGG) backend so PyGeoIpMap can be used 'headless'
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

import geoip2.database

def prueba_geoip2():
    reader = geoip2.database.Reader('GeoLite2-City.mmdb')
    response = reader.city('128.101.101.101')
    print response.country.iso_code
    print response.country.name
    print response.country.names['zh-CN']
    print response.subdivisions.most_specific.name
    print response.subdivisions.most_specific.iso_code
    print response.city.name
    print response.postal.code
    print response.location.latitude
    print response.location.longitude
    reader.close()

prueba_geoip2()
    
def get_ip(ip_file):
    """
    Returns a list of IP addresses from a file containing one IP per line.
    """
    with contextlib.closing(ip_file):
        return [line.strip() for line in ip_file]


def get_lat_lon(ip_list=[], lats=[], lons=[]):
    """
    This function connects to the FreeGeoIP web service to get info from
    a list of IP addresses.
    Returns two lists (latitude and longitude).
    """
    print("Processing {} IPs...".format(len(ip_list)))
    for ip in ip_list:
        r = requests.get("https://freegeoip.net/json/" + ip)
        json_response = r.json()
        print("{ip}, {region_name}, {country_name}, {latitude}, {longitude}".format(**json_response))
        if json_response['latitude'] and json_response['longitude']:
            lats.append(json_response['latitude'])
            lons.append(json_response['longitude'])
    return lats, lons


def geoip_lat_lon(gi, ip_list=[], lats=[], lons=[]):
    """
    This function uses the MaxMind library and databases to geolocate IP addresses
    Returns two lists (latitude and longitude).
    """
    print("Processing {} IPs...".format(len(ip_list)))
    for ip in ip_list:
        try:
            #r = gi.record_by_addr(ip)
            r = gi.city(ip)
        except Exception:
            print("Unable to locate IP: %s" % ip)
            continue
        if r is not None:
            #print("%s {country_iso_code} {latitude}, {longitude}".format(**r) % ip)
            #print("%s {country.iso_code}".format(**r) % ip)
            print "%s %s, %s", r.country.iso_code, r.location.latitude, r.location.longitude
            lats.append(r.location.latitude)
            lons.append(r.location.longitude)
            #lats.append(r['latitude'])
            #lons.append(r['longitude'])
    return lats, lons


def get_lat_lon_from_csv(csv_file, lats=[], lons=[]):
    """
    Retrieves the last two rows of a CSV formatted file to use as latitude
    and longitude.
    Returns two lists (latitudes and longitudes).
    Example CSV file:
    119.80.39.54, Beijing, China, 39.9289, 116.3883
    101.44.1.135, Shanghai, China, 31.0456, 121.3997
    219.144.17.74, Xian, China, 34.2583, 108.9286
    64.27.26.7, Los Angeles, United States, 34.053, -118.2642
    """
    with contextlib.closing(csv_file):
        reader = csv.reader(csv_file)
        for row in reader:
            lats.append(row[-2])
            lons.append(row[-1])

    return lats, lons


def generate_map(output, lats=[], lons=[], wesn=None):
    """
    Using Basemap and the matplotlib toolkit, this function generates a map and
    puts a red dot at the location of every IP addresses found in the list.
    The map is then saved in the file specified in `output`.
    """
    
    plt.clf()
    print("Generating map and saving it to {}".format(output))
    if wesn:
        wesn = [float(i) for i in wesn.split('/')]
        m = Basemap(projection='cyl', resolution='l',
                llcrnrlon=wesn[0], llcrnrlat=wesn[2],
                urcrnrlon=wesn[1], urcrnrlat=wesn[3])
    else:
        m = Basemap(projection='cyl', resolution='l')
    m.bluemarble()
    #m.shadedrelief()
    #m.etopo()
    x, y = m(lons, lats)
    #m.pyplot(x, 30, 40, 40, color='w')
    #x1, y1 = m(lons+30, lats+30)
    #m.quiver(x, y, 30, 30, color='w')
    
    m.plot(x, y, 40, 40, color='y')
    m.scatter(x, y, s=3, color='#ff0000', marker='o', alpha=0.3) #, alpha=0.3
    #m.plot(-34.6033, -58.3816, 52.35, 4.9167, color='y', latlon=False)
    #m.plot(x, y, '-o')
    
    plt.savefig(output, dpi=300, bbox_inches='tight')

lista_trace_nuestro = ['200.89.161.129', '200.89.165.250', '195.22.219.3', '149.3.181.65',
       '129.250.2.227', '129.250.2.19', '129.250.4.141', '129.250.6.8',
       '81.25.207.146', '50.97.18.211', '50.97.18.249', '159.253.158.131',
       '159.253.148.195']
    
gi = geoip2.database.Reader('GeoLite2-City.mmdb')
lats, lons = geoip_lat_lon(gi, lista_trace_nuestro)
generate_map('salida.png', lats, lons) #, wesn='-12/45/30/65'


US
United States
美国
Minnesota
MN
Minneapolis
55414
44.9759
-93.2166
Processing 13 IPs...
%s %s, %s AR -34.6006 -58.4887
%s %s, %s AR -34.6006 -58.4887
%s %s, %s IT 43.1479 12.1097
%s %s, %s IT 43.1479 12.1097
%s %s, %s US 39.6237 -104.8738
%s %s, %s US 39.6237 -104.8738
%s %s, %s US 39.6237 -104.8738
%s %s, %s US 39.6237 -104.8738
%s %s, %s GB 51.5092 -0.0955
%s %s, %s US 47.6062 -122.3321
%s %s, %s US 47.6062 -122.3321
%s %s, %s NL 52.35 4.9167
%s %s, %s NL 52.35 4.9167
Generating map and saving it to salida.png


In [29]:
# Copyright (C) 2005-2016 IP2Location.com
# All Rights Reserved
#
# This library is free software: you can redistribute it and/or
# modify it under the terms of the GNU Lesser General Public
# License as published by the Free Software Foundation, either
# version 3 of the License, or (at your option) any later version.
#
# This library is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
# Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public
# License along with this library; If not, see <http://www.gnu.org/licenses/>.

import os
import IP2Location

database = IP2Location.IP2Location(os.path.join("", "IP2LOCATION-LITE-DB5.BIN"))

rec = database.get_all("19.5.10.1")

print(rec.country_short)
print(rec.country_long)
print(rec.region)
print(rec.city)
print(rec.isp)	
print(rec.latitude)
print(rec.longitude)			
print(rec.domain)
print(rec.zipcode)
print(rec.timezone)
print(rec.netspeed)
print(rec.idd_code)
print(rec.area_code)
print(rec.weather_code)
print(rec.weather_name)
print(rec.mcc)
print(rec.mnc)
print(rec.mobile_brand)
print(rec.elevation)
print(rec.usage_type)
print("\nYou may download the DB24 sample BIN at http://www.ip2location.com/downloads/sample6.bin.db24.zip for full data display.")

US
United States
Michigan
Dearborn
None
42.31152
-83.191322
None
None
None
None
None
None
None
None
None
None
None
None
None

You may download the DB24 sample BIN at http://www.ip2location.com/downloads/sample6.bin.db24.zip for full data display.


In [17]:
#ULTIMA GEOLOCALIZACION

import argparse
import contextlib
import requests
import sys
import csv
import matplotlib
# Anti-Grain Geometry (AGG) backend so PyGeoIpMap can be used 'headless'
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

import os
import IP2Location


def geoip_lat_lon(gi, ip_list=[], lats=[], lons=[]):
    """
    This function uses the MaxMind library and databases to geolocate IP addresses
    Returns two lists (latitude and longitude).
    """
    print("Processing {} IPs...".format(len(ip_list)))
    for ip in ip_list:
        try:
            r = gi.get_all(ip)
        except Exception:
            print("Unable to locate IP: %s" % ip)
            continue
        if r is not None:
            #print ip, r.country_long, r.latitude, r.longitude
            lats.append(r.latitude)
            lons.append(r.longitude)
    return lats, lons

def obtener_paises(ip_list=[]):
    paises_x_ip = {} 
    gi = IP2Location.IP2Location(os.path.join("", "IP2LOCATION-LITE-DB5.BIN"))
    print("Procesando {} IPs...".format(len(ip_list)))
    for ip in ip_list:
        try:
            r = gi.get_all(ip)
        except Exception:
            print("No se puede localizar la IP: %s" % ip)
            continue
        if r is not None:
            paises_x_ip[ip] = r.country_short
    return paises_x_ip

def generate_map(output, lats=[], lons=[], wesn=None):
    """
    Using Basemap and the matplotlib toolkit, this function generates a map and
    puts a red dot at the location of every IP addresses found in the list.
    The map is then saved in the file specified in `output`.
    """
    
    plt.clf()
    print("Generating map and saving it to {}".format(output))
    if wesn:
        wesn = [float(i) for i in wesn.split('/')]
        m = Basemap(projection='cyl', resolution='l',
                llcrnrlon=wesn[0], llcrnrlat=wesn[2],
                urcrnrlon=wesn[1], urcrnrlat=wesn[3])
    else:
        m = Basemap(projection='cyl', resolution='l')
    m.bluemarble()
    x, y = m(lons, lats)
    
    m.plot(x, y, color='y')
    m.scatter(x, y, s=3, color='#ff0000', marker='o', alpha=0.3) #, alpha=0.3
    
    plt.savefig(output, dpi=300, bbox_inches='tight')

lista_trace_nuestro = ['200.89.161.129', '200.89.165.250', '195.22.219.3', '149.3.181.65',
       '129.250.2.227', '129.250.2.19', '129.250.4.141', '129.250.6.8',
       '81.25.207.146', '50.97.18.211', '50.97.18.249', '159.253.158.131',
       '159.253.148.195']
    


def graficar_ruta(ruta, nombre_imagen):
    gi = IP2Location.IP2Location(os.path.join("", "IP2LOCATION-LITE-DB5.BIN"))
    lats, lons = geoip_lat_lon(gi, ruta)
    generate_map(nombre_imagen, lats, lons) #, wesn='-12/45/30/65'

graficar_ruta(lista_trace_nuestro, 'salida.png')

paises_x_ip = obtener_paises(lista_trace_nuestro)

for ip, pais in paises_x_ip.items():
    print ("%s pertenece a %s" % (ip, pais))



Processing 13 IPs...
Generating map and saving it to salida.png
Procesando 13 IPs...
195.22.219.3 pertenece a IT
159.253.158.131 pertenece a NL
50.97.18.249 pertenece a US
159.253.148.195 pertenece a NL
200.89.161.129 pertenece a AR
129.250.4.141 pertenece a US
50.97.18.211 pertenece a US
200.89.165.250 pertenece a AR
129.250.2.227 pertenece a US
81.25.207.146 pertenece a GB
149.3.181.65 pertenece a IT
129.250.2.19 pertenece a US
129.250.6.8 pertenece a US


In [21]:
''%%time
obtener_paises(['4.69.210.222'])

Procesando 1 IPs...
CPU times: user 597 µs, sys: 39 µs, total: 636 µs
Wall time: 583 µs


{u'4.69.210.222': 'US'}

In [1]:
hosts = dict([('udsu.ru', '94.25.116.37'), ('site.u.pereslavl.ru','95.129.137.162'),('www.kstu.kz','85.29.137.233'), ('keu.kz','46.235.86.164'), ('unak.is','130.208.224.102'),('www.fu-berlin.de','160.45.170.10'),('kgiu.kz','195.210.46.14'),('www.uae.ma','159.253.148.195'),('unis.no','158.39.149.250'),('bifrost.is','185.118.33.147'),('birmingham.ac.uk','147.188.218.91')])